In [1]:
%%capture
 # w and b (wandb) for logging
! pip install wandb

# sacremos - for tokenizing
! pip install sacremos

# fairseq - for training and evaluation of the model
! git clone https://github.com/pytorch/fairseq
%cd fairseq
! pip install --editable ./
%cd ..

%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH

In [2]:
import wandb
wandb.login()
# login authorization.

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
!unzip /content/tokenized_kn_sn.zip

Archive:  /content/tokenized_kn_sn.zip
   creating: content/nmt-kn-sn/tokenized.kn-sn/
  inflating: content/nmt-kn-sn/tokenized.kn-sn/valid.kn-sn.kn.idx  
  inflating: content/nmt-kn-sn/tokenized.kn-sn/train.kn-sn.kn.idx  
  inflating: content/nmt-kn-sn/tokenized.kn-sn/valid.kn-sn.sn.idx  
  inflating: content/nmt-kn-sn/tokenized.kn-sn/dict.sn.txt  
  inflating: content/nmt-kn-sn/tokenized.kn-sn/valid.kn-sn.kn.bin  
  inflating: content/nmt-kn-sn/tokenized.kn-sn/preprocess.log  
  inflating: content/nmt-kn-sn/tokenized.kn-sn/test.kn-sn.sn.bin  
  inflating: content/nmt-kn-sn/tokenized.kn-sn/train.kn-sn.sn.bin  
  inflating: content/nmt-kn-sn/tokenized.kn-sn/train.kn-sn.sn.idx  
  inflating: content/nmt-kn-sn/tokenized.kn-sn/valid.kn-sn.sn.bin  
  inflating: content/nmt-kn-sn/tokenized.kn-sn/test.kn-sn.kn.bin  
  inflating: content/nmt-kn-sn/tokenized.kn-sn/test.kn-sn.kn.idx  
  inflating: content/nmt-kn-sn/tokenized.kn-sn/train.kn-sn.kn.bin  
  inflating: content/nmt-kn-sn/tokenized.kn

In [5]:
! fairseq-train /content/content/nmt-kn-sn/tokenized.kn-sn \
--arch transformer \
--dropout 0.1 \
--attention-dropout 0.1 \
--activation-dropout 0.1 \
--encoder-embed-dim 256 \
--encoder-ffn-embed-dim 512 \
--encoder-layers 3 \
--encoder-attention-heads 8 \
--encoder-learned-pos \
--decoder-ffn-embed-dim 512 \
--decoder-layers 3 \
--decoder-attention-heads 8 \
--decoder-learned-pos \
--max-epoch 10 \
--optimizer adam \
--lr 5e-4 \
--batch-size 128 \
--seed 1 \
--wandb-project "Transformer from scratch - 10 March - kn to sn"

2022-03-10 06:17:07 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2022-03-10 06:17:08 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-10 06:17:11 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': 'Transformer from scratch - 10 March - kn to sn', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_

In [6]:
!fairseq-generate /content/content/nmt-kn-sn/tokenized.kn-sn \
  --path /content/checkpoints/checkpoint_best.pt \
  --beam 5 \
  --remove-bpe \
 | grep ^H | LC_ALL=C sort -V | cut -f3- > test_sn_generated_scratch_transformer.txt 

2022-03-10 06:19:07 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-10 06:19:09 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_p

In [8]:
!fairseq-generate /content/content/nmt-kn-sn/tokenized.kn-sn \
  --path /content/checkpoints/checkpoint_best.pt \
  --beam 5 \
   --remove-bpe \
  | grep ^T | LC_ALL=C sort -V | cut -f2- > test_sn_actual_scratch_transformer.txt 

2022-03-10 06:19:51 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-10 06:19:53 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_p

In [9]:
!fairseq-score --sys /content/test_sn_generated_scratch_transformer.txt --ref /content/test_sn_actual_scratch_transformer.txt

2022-03-10 06:20:21 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
Namespace(ignore_case=False, order=4, ref='/content/test_sn_actual_scratch_transformer.txt', sacrebleu=False, sentence_bleu=False, sys='/content/test_sn_generated_scratch_transformer.txt')
BLEU4 = 57.44, 69.4/58.5/53.9/49.9 (BP=1.000, ratio=1.058, syslen=10669, reflen=10086)


In [10]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
f = open("/content/test_sn_generated_scratch_transformer.txt")
ml_candidates = [i.strip() for i in f.readlines()]
f = open("/content/test_sn_actual_scratch_transformer.txt")
ml_gold = [i.strip() for i in f.readlines()]
total_four = 0
for i in range(len(ml_candidates)):
    candidate = ml_candidates[i].split(" ")
    references = ml_gold[i].split(" ")
    reference = [references]
    score_cumulative4 = sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25))
    total_four = total_four + score_cumulative4
print(total_four/len(ml_candidates))

0.8123600843416234


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!cp /content/checkpoints/checkpoint_best.pt /content/drive/MyDrive/Transformer_from_scratch_sn/